In [134]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from spacy.lang.en import English
import spacy

text = open('./fake-corpus.txt').read().lower()
nlp = spacy.load('en')

print(type(nlp))

doc = nlp(text)

print(type(doc))

original_doc_list = text.splitlines()

<class 'spacy.lang.en.English'>
<class 'spacy.tokens.doc.Doc'>


In [136]:
docs = []
next_doc = []

for w in doc:
    if w.text != '\n' and not w.is_stop and not w.is_punct and not w.like_num:
        next_doc.append(w.lemma_)
    if w.text == '\n':
        docs.append(next_doc)
        next_doc = []
        
print(docs[3])
print(original_doc_list[3])

['like', 'animal', 'like', 'cat', 'dog']
i like animals like cats and dogs


In [137]:
from gensim import corpora
from gensim.models.phrases import Phrases, Phraser

bigram_model = Phrases(docs)
docs = [bigram_model[doc] for doc in docs]
docs = [bigram_model[doc] for doc in docs]

In [138]:
dictionary = corpora.Dictionary(docs)
corpus = [dictionary.doc2bow(doc) for doc in docs]

In [139]:
from gensim.models import TfidfModel

tfidf = TfidfModel(corpus)

# corpus = [tfidf[doc] for doc in corpus]

In [140]:
from gensim.models import LsiModel, LdaModel

model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=2)

In [141]:
import pandas as pd
from IPython.display import display, HTML

doc_id = 8

doc_as_bow = corpus[doc_id]

def get_top_terms(topic_id):
    terms = model.get_topic_terms(topicid=topic_id, topn=5)
    termids = map(lambda x: x[0], terms)
    words = map(lambda x: dictionary[x], termids)
    return list(words)

tts = model.get_document_topics(bow=doc_as_bow)

ls = list(map(list, list(zip(*tts))))
t_ids = ls[0]
t_probs = ls[1]
t_wordlist = list(map(get_top_terms, t_ids))

print(original_doc_list[doc_id])

df = pd.DataFrame()
df['topic id'] = pd.Series(t_ids)
df['probability'] = pd.Series(t_probs)
df['topic terms'] = pd.Series(t_wordlist)

display(df)

it is raining cats and dogs


,topic id,probability,topic terms
0,0,0.143841,"[dog, rain, cat, like, icecream]"
1,1,0.856159,"[cat, dog, rain, like, topic]"


In [142]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(model, corpus, dictionary)

PreparedData(topic_coordinates=              x    y  topics  cluster       Freq
topic                                           
1      0.053418  0.0       1        1  67.381645
0     -0.053418  0.0       2        1  32.618351, topic_info=     Category       Freq        Term      Total  loglift  logprob
term                                                             
10    Default   3.000000    icecream   3.000000  28.0000  28.0000
8     Default   3.000000     vanilla   3.000000  27.0000  27.0000
3     Default   6.000000        like   6.000000  26.0000  26.0000
6     Default   2.000000   chocolate   2.000000  25.0000  25.0000
22    Default   1.000000       tooth   1.000000  24.0000  24.0000
21    Default   1.000000         not   1.000000  23.0000  23.0000
20    Default   1.000000        hurt   1.000000  22.0000  22.0000
18    Default   1.000000    possible   1.000000  21.0000  21.0000
19    Default   1.000000    superior   1.000000  20.0000  20.0000
26    Default   1.000000         pet   1.000000  19.0000  19.0000
25    Default   1.000000      people   1.000000  18.0000  18.0000
7     Default   1.000000      flavor   1.000000  17.0000  17.0000
24    Default   1.000000        have   1.000000  16.0000  16.0000
27    Default   1.000000      prefer   1.000000  15.0000  15.0000
16    Default   2.000000       clear   2.000000  14.0000  14.0000
2     Default   2.000000         ice   2.000000  13.0000  13.0000
0     Default   2.000000       cream   2.000000  12.0000  12.0000
1     Default   1.000000     discuss   1.000000  11.0000  11.0000
12    Default   1.000000  particular   1.000000  10.0000  10.0000
11    Default   1.000000     obvious   1.000000   9.0000   9.0000
4     Default   3.000000       topic   3.000000   8.0000   8.0000
5     Default   1.000000       would   1.000000   7.0000   7.0000
14    Default  28.000000         cat  28.000000   6.0000   6.0000
17    Default   1.000000    emphasis   1.000000   5.0000   5.0000
9     Default   2.000000      corpus   2.000000   4.0000   4.0000
13    Default   2.000000      animal   2.000000   3.0000   3.0000
23    Default  26.000000        rain  26.000000   2.0000   2.0000
15    Default  29.000000         dog  29.000000   1.0000   1.0000
14     Topic1  24.398748         cat  28.584053   0.2365  -1.3805
23     Topic1  22.010920        rain  26.336908   0.2154  -1.4835
...       ...        ...         ...        ...      ...      ...
21     Topic1   0.469920         not   1.924776  -1.0152  -5.3302
22     Topic1   0.445745       tooth   1.933969  -1.0728  -5.3830
22     Topic2   1.488225       tooth   1.933969   0.8583  -3.4519
21     Topic2   1.454856         not   1.924776   0.8404  -3.4746
20     Topic2   1.451445        hurt   1.923836   0.8385  -3.4770
18     Topic2   1.408855    possible   1.912102   0.8149  -3.5067
19     Topic2   1.359919    superior   1.898619   0.7866  -3.5421
10     Topic2   2.663791    icecream   3.781796   0.7698  -2.8698
8      Topic2   2.658663     vanilla   3.780384   0.7683  -2.8717
6      Topic2   1.883855   chocolate   2.804942   0.7222  -3.2162
26     Topic2   1.096178         pet   1.825955   0.6100  -3.7577
25     Topic2   1.095153      people   1.825672   0.6092  -3.7586
7      Topic2   1.074954      flavor   1.820107   0.5937  -3.7772
24     Topic2   1.050683        have   1.813420   0.5745  -3.8001
27     Topic2   0.941357      prefer   1.783299   0.4814  -3.9100
3      Topic2   3.027778        like   6.167993   0.4088  -2.7417
1      Topic2   0.805608     discuss   1.745898   0.3469  -4.0657
12     Topic2   0.797747  particular   1.743732   0.3383  -4.0755
11     Topic2   0.784027     obvious   1.739952   0.3231  -4.0928
16     Topic2   1.154066       clear   2.603874   0.3066  -3.7062
2      Topic2   1.133174         ice   2.598118   0.2905  -3.7245
5      Topic2   0.754348       would   1.731776   0.2892  -4.1314
0      Topic2   1.120453       cream   2.594614   0.2806  -3.7358
17     Topic2   0.666856    emphasis   1.707670   0.1800  -4.2547